In [56]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [15]:
import nltk
from nltk.corpus import stopwords

from tqdm import tqdm
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from sklearn import utils

In [76]:
path = '../input/imdb-50k-movie-reviews-test-your-bert/'

train_data = pd.read_csv(path + 'train.csv')
test_data = pd.read_csv(path + 'test.csv')



In [5]:
data = pd.read_csv("./data.csv", sep="|")

In [11]:
data.rename(columns={"review":"text"}, inplace=True)
train_data, test_data = train_test_split(data, test_size = 0.25)

In [77]:
# creating a new column containing either 1 or 0 (1 for pos and 0 for neg)

train_data["sent"] = train_data["sentiment"].apply(lambda x: 1 if x =="pos" else 0)
test_data["sent"] = test_data["sentiment"].apply(lambda x: 1 if x =="pos" else 0)

In [3]:
train_data[train_data["sentiment"] == "pos"].text.str.len().describe()

count    12500.000000
mean      1347.160240
std       1046.747365
min         70.000000
25%        695.000000
50%        982.000000
75%       1651.000000
max      13704.000000
Name: text, dtype: float64

In [ ]:
train_data[train_data["sentiment"] == "neg"].text.str.len().describe()

In [18]:
# fonction that tokenizes a document

def tokenize_document(doc):
    tokens = []
    for sent in nltk.sent_tokenize(doc):
        for word in nltk.word_tokenize(sent):
            if len(word) > 2 and word not in stop_words :
                tokens.append(word)
    return tokens        

In [17]:
stop_words = stopwords.words('english')

In [19]:
# Creating a new columns with tokens
train_data["token"] = train_data["text"].apply(lambda x:tokenize_document(x))
test_data["token"] = test_data["text"].apply(lambda x:tokenize_document(x))

<ipython-input-19-d907a101efe1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["token"] = train_data["text"].apply(lambda x:tokenize_document(x))
<ipython-input-19-d907a101efe1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["token"] = test_data["text"].apply(lambda x:tokenize_document(x))


In [20]:
train_data

,text,sent,link,token
17049,Finally. A fresh movie with complete originali...,1,https://www.imdb.com/title/tt7131622/reviews?r...,"[Finally, fresh, movie, complete, originality,..."
22926,As a avid gamer and love the uncharted games w...,0,https://www.imdb.com/title/tt1464335/reviews?r...,"[avid, gamer, love, uncharted, games, ive, pla..."
2493,Recently I watched this very exciting Drama mo...,1,https://www.imdb.com/title/tt0111161/reviews?r...,"[Recently, watched, exciting, Drama, movie, Th..."
18199,This movie was just so king and wonderful! Its...,1,https://www.imdb.com/title/tt6467266/reviews?r...,"[This, movie, king, wonderful, Its, good, 1st,..."
16953,Disappointed. Didn't get it.\nI expected more ...,0,https://www.imdb.com/title/tt7131622/reviews?r...,"[Disappointed, Did, n't, get, expected, Tarant..."
...,...,...,...,...
7760,I wanted to like this movie but just couldn't....,0,https://www.imdb.com/title/tt0120737/reviews?r...,"[wanted, like, movie, could, n't, Yea, epic, A..."
25849,In an age where all blockbusters look the same...,1,https://www.imdb.com/title/tt5834426/reviews?r...,"[age, blockbusters, look, everything, nothing,..."
4999,"It's over.It's over. And now, somehow, nothing...",1,https://www.imdb.com/title/tt0167260/reviews?r...,"[over.It, And, somehow, nothing, ever, again.T..."
3853,This film is very beautiful and all the actors...,1,https://www.imdb.com/title/tt0068646/reviews?r...,"[This, film, beautiful, actors, wonderful]"


In [21]:
# tagging each document with its flag
tagged_doc = train_data.apply(lambda x: TaggedDocument(words=x['token'], tags=[x["sent"]]), axis=1)
tagged_doc_test = test_data.apply(lambda x: TaggedDocument(words=x['token'], tags=[x["sent"]]), axis=1)

In [76]:
tagged_doc.iloc[0]

TaggedDocument(words=['Now', "n't", 'deny', 'purchased', 'eBay', 'high', 'expectations', 'This', 'incredible', 'out-of-print', 'work', 'master', 'comedy', 'enjoy', 'However', 'soon', 'disappointed', 'Apologies', 'enjoyed', 'found', 'Compleat', 'difficult', 'watch', 'got', 'smiles', 'sure', 'majority', 'funny', 'came', 'music', 'videos', "'ve", 'got', 'DVD', 'rest', 'basically', 'filler', 'You', 'could', 'tell', 'greatest', 'video', 'achievement', 'honor', 'goes', 'UHF', 'Honestly', 'doubt', 'ever', 'make', 'jump', 'DVD', "'re", 'ultra-hardcore', 'fan', 'HAVE', 'everything', 'buy', 'tape', 'eBay', 'Just', "n't", 'pay', 'much'], tags=['neg'])

In [22]:
# Getting the numbre of cores in the machine
import multiprocessing
cores = multiprocessing.cpu_count()
cores

4

In [23]:
# creating the model and building the vocabulary
model_dbow = Doc2Vec(dm=1, vector_size=100,  min_count=2, sample = 0, workers=cores, epochs=30)
model_dbow.build_vocab([x for x in tqdm(tagged_doc.values)])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 19969/19969 [00:00<00:00, 1282340.30it/s]


In [24]:
# train the model for 30 epochs ( this cell may take some time to execute)

model_dbow.train(tagged_doc.values, total_examples=len(tagged_doc.values), epochs=model_dbow.epochs)

In [25]:
# exemple of the embedding of the first document in the dataset
model_dbow.infer_vector(tagged_doc[0].words)

array([-1.1709553 , -1.2236848 , -0.60866165, -0.5273259 ,  0.4867903 ,
       -1.2583244 , -0.24497163,  1.1574725 ,  0.36740276,  0.45968094,
        0.963981  ,  0.42612943, -0.243746  , -0.49975955, -0.04492902,
       -0.7804698 ,  0.49073207, -1.2196075 , -0.13684276,  0.7558496 ,
        1.4321141 ,  1.245945  , -0.03449751, -0.06608846, -0.8910699 ,
       -0.31143066, -0.7566022 ,  0.64676625,  0.05431699, -1.170472  ,
        1.6258638 , -0.07204605, -0.1961475 , -0.791918  , -0.36986396,
        0.6907102 , -0.9268505 , -0.30342692, -1.3098334 ,  0.3127035 ,
        0.3765506 , -2.4879196 ,  0.4357923 , -0.80003667, -0.41073608,
        0.26416782, -0.6323895 ,  0.9565924 ,  0.89233375, -0.279084  ,
        0.19824275, -0.25560838, -0.5945026 , -1.2474824 , -0.8367374 ,
        0.8691495 , -1.0395129 ,  1.0500854 , -0.6140938 ,  0.9891239 ,
        0.3096577 ,  1.1555976 , -0.0550534 , -0.8887782 , -1.4238105 ,
        0.628893  , -1.1558276 , -0.6255648 ,  0.71179825, -0.72

In [26]:
# storing the embedding in our dataframe
train_data["embedding"] = train_data["token"].apply(lambda x: model_dbow.infer_vector(x))
test_data["embedding"] = test_data["token"].apply(lambda x: model_dbow.infer_vector(x))

<ipython-input-26-8762267e8775>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["embedding"] = train_data["token"].apply(lambda x: model_dbow.infer_vector(x))
<ipython-input-26-8762267e8775>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["embedding"] = test_data["token"].apply(lambda x: model_dbow.infer_vector(x))


In [29]:
# A function to put the embeddings and the target in a proper form
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, regressors

In [30]:
y, x = vec_for_learning(model_dbow, tagged_doc)

In [32]:
y_test, x_test = vec_for_learning(model_dbow, tagged_doc_test)

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report


In [34]:
# creating a logistic regression model with "sag" solver since it's recommended for large datasets
model = LogisticRegression(penalty="l2", solver="sag")
model.fit(x, y)

LogisticRegression(solver='sag')

In [35]:
# predicting the sentiments of test data
y_pred = model.predict(x_test)

#printing accuracy score
print(accuracy_score(y_test, y_pred))

0.8802764007811327


In [185]:
# a more detailed report 
print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

         neg       0.81      0.85      0.83     12500
         pos       0.84      0.80      0.82     12500

    accuracy                           0.82     25000
   macro avg       0.83      0.82      0.82     25000
weighted avg       0.83      0.82      0.82     25000



In [11]:
### Using the score of words to classify reviews

In [36]:
from nltk.corpus import sentiwordnet as swn

In [37]:
nltk_to_sentiwordnet = {
    "NN": "n",
    "VB": "v",
    "JJ": "a",
    "RB": "r",
}

In [44]:
nltk.download('wordnet')
nltk.download('sentiwordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\root\AppData\Roaming\nltk_data...
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\root\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\root\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [39]:
stop_words = set(stopwords.words("english"))

In [45]:
def get_doc_sentiment(doc):
    
    sentences = nltk.sent_tokenize(doc)
    words_token = [nltk.word_tokenize(sentence) for sentence in sentences]
    tagged_words = [nltk.pos_tag(word_token) for word_token in words_token]
    l = [element for lists in tagged_words for element in lists ]
    l = [word for word in l if word not in stop_words]
    pos_score = []
    neg_score = []
    neut_score = []
    
    for word, pos in l:
        pos_snt = nltk_to_sentiwordnet.get(pos,None) 
        
        if pos_snt is None:
            continue
        synsets = swn.senti_synsets(word.lower(), pos=pos_snt)
        synsets = [synset for synset in synsets]
        for syn in synsets[:1]:
            pos_score.append(syn.pos_score())
            neg_score.append(syn.neg_score())
            neut_score.append(1-syn.obj_score())
            
        
    return 1 if  np.average(pos_score) > np.average(neg_score) else 0
get_doc_sentiment(train_data["text"].iloc[0])

1

In [46]:
train_data["predicted_label"] = train_data["text"].apply(lambda x: get_doc_sentiment(x))

C:\Users\root\anaconda3\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
<ipython-input-46-bfa97b19e7e9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["predicted_label"] = train_data["text"].apply(lambda x: get_doc_sentiment(x))


In [47]:
train_data[train_data["sent"] == train_data["predicted_label"]].shape[0]/train_data.shape[0]

0.6241674595623216

In [48]:
test_data["predicted_label"] = test_data["text"].apply(lambda x: get_doc_sentiment(x))

C:\Users\root\anaconda3\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
<ipython-input-48-66471af0d23c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["predicted_label"] = test_data["text"].apply(lambda x: get_doc_sentiment(x))


In [49]:
test_data[test_data["sent"] == test_data["predicted_label"]].shape[0]/test_data.shape[0]

0.6199489259426167

In [50]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\root\AppData\Roaming\nltk_data...


True

In [51]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [52]:
print(get_doc_sentiment(train_data["text"].iloc[0]))
sid.polarity_scores(train_data["text"].iloc[0])

1


{'neg': 0.045, 'neu': 0.736, 'pos': 0.219, 'compound': 0.9299}

In [53]:
train_data["predicted_label_vader"] = train_data["text"].apply(lambda x: sid.polarity_scores(x)["compound"])
train_data.loc[train_data["predicted_label_vader"] < 0, ["predicted_label_vader"]] = 0
train_data.loc[train_data["predicted_label_vader"] > 0, ["predicted_label_vader"]] = 1
train_data["predicted_label_vader"] = train_data["predicted_label_vader"].astype(int)

<ipython-input-53-2aed1723481f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["predicted_label_vader"] = train_data["text"].apply(lambda x: sid.polarity_scores(x)["compound"])
C:\Users\root\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\root\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [54]:
test_data["predicted_label_vader"] = test_data["text"].apply(lambda x: sid.polarity_scores(x)["compound"])
test_data.loc[test_data["predicted_label_vader"] < 0, ["predicted_label_vader"]] = 0
test_data.loc[test_data["predicted_label_vader"] > 0, ["predicted_label_vader"]] = 1
test_data["predicted_label_vader"] = test_data["predicted_label_vader"].astype(int)

<ipython-input-54-a52eb5848f25>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["predicted_label_vader"] = test_data["text"].apply(lambda x: sid.polarity_scores(x)["compound"])
C:\Users\root\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\root\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [55]:
print(test_data[test_data["sent"] == test_data["predicted_label_vader"]].shape[0]/test_data.shape[0]) 
train_data[train_data["sent"] == train_data["predicted_label_vader"]].shape[0]/train_data.shape[0]

0.7531921285864503


0.7542190395112425